<img src="https://www.epfl.ch/about/overview/wp-content/uploads/2020/07/logo-epfl-1024x576.png" style="padding-right:10px;width:140px;float:left"></td>
<h2 style="white-space: nowrap">Image Processing Laboratory Notebooks</h2>
<hr style="clear:both">
<p style="font-size:0.85em; margin:2px; text-align:justify">
This Juypter notebook is part of a series of computer laboratories which are designed
to teach image-processing programming; they are running on the EPFL's Noto server. They are the practical complement of the theoretical lectures of the EPFL's Master course <b>Image Processing I</b> 
(<a href="https://moodle.epfl.ch/course/view.php?id=522">MICRO-511</a>) taught by Prof. M. Unser and Prof. D. Van de Ville.
</p>
<p style="font-size:0.85em; margin:2px; text-align:justify">
The project is funded by the Center for Digital Education and the School of Engineering. It is owned by the <a href="http://bigwww.epfl.ch/">Biomedical Imaging Group</a>. 
The distribution or the reproduction of the notebook is strictly prohibited without the written consent of the authors.  &copy; EPFL 2020.
</p>
<p style="font-size:0.85em; margin:0px"><b>Authors</b>: 
    <a href="mailto:pol.delaguilapla@epfl.ch">Pol del Aguila Pla</a>, 
    <a href="mailto:kay.lachler@epfl.ch">Kay Lächler</a>,
    <a href="mailto:alejandro.nogueronaramburu@epfl.ch">Alejandro Noguerón Arámburu</a>, and
    <a href="mailto:daniel.sage@epfl.ch">Daniel Sage</a>.
</p>
<hr style="clear:both">
<h1>Lab 2: Digital filtering</h1>
<div style="background-color:#F0F0F0;padding:4px">
    <p style="margin:4px;"><b>Released</b>: Thursday November 5, 2020</p>
    <p style="margin:4px;"><b>Submission</b>: <span style="color:red">Friday November 13, 2020</span> (before 11:59PM) on <a href="https://moodle.epfl.ch/course/view.php?id=522">Moodle</a></p>
    <p style="margin:4px;"><b>Grade weigth</b>: 10% of the overall grade</p>
    <p style="margin:4px;"><b>Remote help</b>: Monday November 9, on Zoom (see Moodle for link and time)</p>    
    <p style="margin:4px;"><b>Related lectures</b>: Chapter 3</p>
</div>

### Student Name: 
### SCIPER: 

Double-click on this cell and fill your name and SCIPER number. Then, run the cell below to verify your identity in Noto and set the seed for random results.

In [1]:
%use sos
import getpass
# This line recovers your camipro number to mark the images with your ID
uid = int(getpass.getuser().split('-')[2]) if len(getpass.getuser().split('-')) > 2 else ord(getpass.getuser()[0])
print(f'SCIPER: {uid}')

SCIPER: 107


### <a name="imports_"></a> Imports
In the next cell we import the python libraries that we will use throughout the lab, as well as the `IPLabViewer()` class for interactive image visualization based on the `ipywidgets` library:
* [`matplotlib.pyplot`](https://matplotlib.org), to display images
* [`ipywidgets`](https://ipywidgets.readthedocs.io/en/latest/), to make the image display interactive
* [`numpy`](https://numpy.org/doc/stable/reference/index.html), for mathematical operations on arrays
* [`openCV` (cv2)](https://docs.opencv.org/2.4/index.html), for image-processing tasks
* [`scipy.ndimage`](https://docs.scipy.org/doc/scipy/reference/ndimage.html), Scipy's specific module for multidimensional image processing
* [`scikit-image` (skimage)](https://scikit-image.org/docs/stable/api/api.html), also for image-processing tasks

We will then load the `IPLabViewer` class (either see the complete documentation [here](https://github.com/Biomedical-Imaging-Group/IPLabImageViewer/wiki/Python-IPLabViewer()-Class), run the python command `help(viewer)` after loading the class, or refer to [Lab 0: Introduction](./Introductory.ipynb)).

Finally, we load the images you will use in the exercise to test your algorithms. 

In [2]:
%use sos
# Configure plotting as dynamic
%matplotlib widget

# Import standard required packages for this exercise
import matplotlib.pyplot as plt
import numpy as np
import cv2 as cv 
import scipy.ndimage as ndi
import ipywidgets as widgets
import skimage
from skimage import filters
from skimage import io

# Import IPLabViewer() Class
from lib.iplabs import IPLabViewer as viewer

# Load images to be used in this exercise 
bikesgray = plt.imread('images/bikesgray.tif')
camera = plt.imread('images/camera-16bits.tif')
spots = cv.imread('images/spots.tif',cv.IMREAD_UNCHANGED)

In [3]:
%use javascript
// import IPLabImageAccess as Image
var Image = require('./lib/IPLabImageAccess.js')

# <a name="filtering_lab"></a> Digital filtering (15 Points)

In this lab you will review the basics of digital image filtering. You will review topics such as separable filters, non-separable filters, filter design, and convolution. Finally, you will program a spot detector using digital filtering techniques.

## Index
1. [Edge detection](#-1.-Edge-detection-(3-points))
    1. [Non-separable version](#-1.A.-Non-separable-version-(1-point)) (**1 point**)
    2. [Separable version](#-1.B.-Separable-version-(2-points)) (**2 points**)
    3. [Comparison: Separable vs Non-separable](#-1.C.-Comparison:-Separable-vs-Non-separable) 
    4. [Edge detection in Python](#-1.D.-Edge-detection-in-Python)
2. [Implementation and classification of digital filters](#-2.-Implementation-and-classification-of-digital-filters-(3-points)) (**3 points**)
    1. [Mask A](#-2.A.-Mask-A)
    2. [Mask B](#-2.B.-Mask-B)
    3. [Mask C](#-2.C.-Mask-C)
    4. [Mask D](#-2.D.-Mask-D)
    5. [Mask E](#-2.E.-Mask-E)
    6. [Mask F](#-2.F.-Mask-F)
3. [Gaussian filter](#-3.-Gaussian-filter-(4-points))
    1. [Implementation of a 2D Gaussian filter](#-3.A.-Implementation-of-a-2D-Gaussian-filter-(4-points)) (**4 points**)
    2. [Gaussian filter in Python](#-3.B.-Gaussian-filter-in-Python)
4. [Application: Spot detector](#-4.-Application:-Spot-detector-(5-points))
    1. [Difference of Gaussians](#-4.A.-Difference-of-Gaussians-(2-points)) (**2 points**)
    2. [Local maxima](#-4.B.-Local-maxima-(1-point)) (**1 point**)
    3. [Spot detector](#-4.C.-Spot-detector-(2-points)) (**2 points**)

In particular, you will: 
* Fully understand the implementation of a digital filter (Sections 1 and 2);
* Learn how to implement filters using different image-processing libraries in Python (Sections 1 and 2);
* Implement a method `gaussian()` for separable Gaussian filtering (Section 3); and
* Develop a practical application of filtering (Section 4).

The goal is for you to fully understand the filtering algorithm at a pixel-by-pixel level (implementing it in JavaScript), and at the same time to learn the standard tools to use filtering in professional image-processing libraries in Python.

<div class=" alert alert-danger">

<b>Important:</b> Each cell that contains code begins with `%use sos` or `%use javascript`. This indicates if the code in this specific cell should be written in Python or JavaScript. Do not change or remove any lines of code that begin with an %. They are used for the notebook to run smoothly with <code>SoS</code> and need to be on the first line of each cell!
</div>

Good luck and enjoy! 

### Visualize images
Get familiar now with the images you are going to be using.

Remember that to use the `IPLabViewer` class, you only need to call it with an image (make sure that the image is a `numpy.ndarray`, or a list of such arrays). From there, you can click the button `Show Widgets` and change the plotting range, visualize the histogram, get the statistics, etc.

In [4]:
%use sos

# Declare image_list for ImageViewer
image_list = [bikesgray, camera, spots]

imgs_viewer = viewer(image_list, widgets = True, hist = True)

# <a class="anchor"></a> 1. Edge detection (3 points)

As you might remember from the course, edge detection algorithms try to outline steep intesity differences between neighbouring pixels. A common vertical edge detector (detecting changes in the $x$ direction) is the filter represented by the impulse response 

$$h[m,n] = \begin{bmatrix} -1 & 0 & 1 \\ -2 & \boxed{0} & 2 \\ -1 & 0 & 1 \end{bmatrix}\,.$$

If pixels neighbouring a given pixel position all have a similar value, the output at that pixel will have a value close to zero after applying the mask. On the other hand, if there is a big difference in the values of the left and right columns, this likely indicates some kind of vertical edge and consequently the output at that location will have either a highly positive or highly negative intensity. 

## <a class="anchor"></a> 1.A. Non-separable version (1 point)

In the next cell we provide the method `filterNonSeparable()`. **This function is supposed to perform a convolution** on two input images `img` and `mask`. However, whoever implemented the function (_it certainly wasn't us_) had not heard about the difference between a correlation and a convolution. Long story short, the function does not provide the correct result. **For 1 point**, find the mistake and correct it in the cell below.

<div class="alert alert-info">

**Hint:** The two-dimensional convolution between two images `h` and `f` is given by
$$(h \ast f)[x,y] = \sum_{m \in \mathbb{Z}}\sum_{n \in \mathbb{Z}}f[m,n]h[x-m,y-n]$$
</div>

In [5]:
%use javascript

// function that performs a convolution on the two input parameters img and mask
function filterNonSeparable(img, mask){
    // create output Image, of the same shape as the input
    var output = new Image(img.shape());
    // iterate through every pixel
    for(var x = 0; x < img.nx; x++){  
        for(var y = 0; y < img.ny; y++){
            // get neighbourhood with the size of mask of current position
            var neigh = img.getNbh(x, y, mask.nx, mask.ny)
            // perform convolution
            var val = 0;
            for(var k = 0; k < mask.nx; k++){
                for(var l = 0; l < mask.ny; l++){
                    val += neigh.getPixel(k, l) * mask.getPixel(k, l);
                }
            }
            // assign value on output image
            output.setPixel(x, y, val)
        }
    }
    return output
}

Run the next cell to perform a sanity check (convolution of the mask with an impulse image).

In [6]:
// Lets do a sanity check
// we use a simple 3x3 impulse image
var impulse = new Image([[0, 0, 0], [0, 1, 0], [0, 0, 0]]);
// declare mask
var mask = new Image([[-1, 0, 1],[-2, 0, 2],[-1, 0, 1]]);
// check the result
console.log('Convolving by an impulse should return the unchanged mask.\nMask:\t', mask.toArray() )
console.log('Result:\t', filterNonSeparable(impulse, mask).toArray());
if(filterNonSeparable(impulse, mask).imageCompare(mask) == false){
    throw new Error('Sorry, the function still provides an incorrect output.')
}
console.log('Well done, now the function probably performs a convolution!')

Convolving by an impulse should return the unchanged mask.
Mask:	 [ [ -1, 0, 1 ], [ -2, 0, 2 ], [ -1, 0, 1 ] ]
Result:	 [ [ 1, 0, -1 ], [ 2, 0, -2 ], [ 1, 0, -1 ] ]


Error: Sorry, the function still provides an incorrect output.

### Applying the function

Run the next cell to apply the `filterNonSeparable` function with the mask given above to the images `bikesgray`, `camera` and `spots`.

In [7]:
%use javascript
%get bikesgray camera spots
%put vedge_bike_nons vedge_camera_nons vedge_spots_nons

// apply Non separable filter
var vedge_bike_nons = filterNonSeparable(new Image(bikesgray), mask).toArray();
var vedge_spots_nons = filterNonSeparable(new Image(spots), mask).toArray();
var vedge_camera_nons = filterNonSeparable(new Image(camera), mask).toArray();
//### BEGIN HIDDEN TESTS
// declare mask
var mask = new Image([[-1, 0, 1],[-2, 0, 2],[-1, 0, 1]]);

// function that performs a convolution on the two input parameters img and mask
function filterNonSeparableCorrect(img, mask){
    // create output Image, of the same shape as the input
    var output = new Image(img.shape());
    // iterate through every pixel
    for(var x = 0; x < img.nx; x++){  
        for(var y = 0; y < img.ny; y++){
            // get neighbourhood with the size of mask of current position
            var neigh = img.getNbh(x, y, mask.nx, mask.ny)
            // perform convolution
            var val = 0;
            for(var k = 0; k < mask.nx; k++){
                for(var l = 0; l < mask.ny; l++){
                    val += neigh.getPixel(k, l) * mask.getPixel(mask.nx - 1 - k, mask.ny - 1 - l);
                }
            }
            // assign value on output image
            output.setPixel(x, y, val)
        }
    }
    return output
}
// declare mask
var mask = new Image([[-1, 0, 1],[-2, 0, 2],[-1, 0, 1]]);
// run the functions
var vedge_bike_nons_check = filterNonSeparable(new Image(bikesgray), mask);
var vedge_bike_nons_correct = filterNonSeparableCorrect(new Image(bikesgray), mask);
// check the result
if(vedge_bike_nons_check.imageCompare(vedge_bike_nons_correct) == false){
    throw new Error('The filterNonSeparable function does not provide the correct output.')
}
//### END HIDDEN TESTS

Error: The filterNonSeparable function does not provide the correct output.

### Visualize the results
We will use the `IPLabViewer` to take a look at the results of the edge detection. Run the next cell to visualize the results and use the `Next` / `Prev` buttons to cycle through the images.

In [8]:
%use sos
%get vedge_bike_nons vedge_camera_nons vedge_spots_nons --from javascript

# Declare parameters for viewer
image_list_nons = [vedge_bike_nons, vedge_camera_nons, vedge_spots_nons]
title_list = ['Bike - Vertical edges', 'Camera - Vertical edges', 'Spots - Vertical edges']

# Make sure that the images are numpy arrays and not lists
for image in range(len(image_list_nons)):
    image_list_nons[image] = np.array(image_list_nons[image])

vedge_nons_viewer = viewer(image_list_nons, title = title_list, widgets=True)

## <a class="anchor"></a> 1.B. Separable version (2 points)

In this subsection, you are going to modify the function `filterSeparable()` we provide to implement the separable version of the filter. This function is based on the method `filter1D()` (also declared in the next cell), which takes as input a row/column of the image and a 1D mask and performs a 1D convolution. Recall that separable filters are divided into two 1D masks (`mx` and `my` in the code below). 

In the next cell, **for 1 point** complete the method `filterSeparable()` by implementing the column filtering. The row filtering is already coded and can be used as a reference.

<div class="alert alert-info">
    
**Note:** `img.getRow(y)` extracts the $y^{\text{th}}$ row from `img` while `output.putRow(y, new_row)` inserts `new_row` into the $y^{\text{th}}$ row of `output`. Similarly you can use `.getColumn(x)` and `.putColumn(x, new_column)` to extract and insert columns from/to `Image` objects.
</div>
<div class="alert alert-info">
    
**Hint:** If you don't remember how a separable filter works, review the theory in your course notes.
</div>

In [71]:
%use javascript

// function that performs a separable convolution on img using two 1D masks mx and my
function filterSeparable(img, mx, my){
    // initialize output Image, of the same shape as the input
    var output = new Image(img.shape());
    
    // iterate through every row 
    for(var y = 0; y < img.ny; y++){
        // extract row
        var row = img.getRow(y);
        // apply filter
        var new_row = filter1D(row, mx)
        // set column in output variable
        output.putRow(y, new_row);    
    }
    // iterate through every column
    for(var x = 0; x < img.nx; x++){
        // filter the columns
        
        // BEGIN SOLUTION
        // extract column
        var column = output.getColumn(x);
        // apply filter
        var new_column = filter1D(column, my); 
        // set column in output variable
        output.putColumn(x, new_column);
        // END SOLUTION
        
    }
    return output
}

// function that applies a 1D filter
function filter1D(img, mask){
    // transpose the input variables if necessary
    if(img.nx == 1){
        img.transposeImage();
    }
    if(mask.nx == 1){
        mask.transposeImage();
    }
    // create the output image
    var output = new Image(img.shape());
    // iterate through all pixels
    for(var x = 0; x < img.nx; x++){
        // get the neighbourhood around position x
        var neigh = img.getNbh(x, 0, mask.nx, 1);
        // declare a variable to store the values of the convolution. 
        var val = 0;
        // iterate through the neighbourhood
        for(var i = 0; i < neigh.nx; i++){
            // perform convolution
            val += neigh.getPixel(i, 0) * mask.getPixel(mask.nx - 1 - i, 0);
        }
        // set value in output array
        output.setPixel(x, 0, val);
    }
    return output
}

### 1D masks

In the cell below, **for 1 point** change the masks `mask_x` and `mask_y` to the correct 1D masks corresponding to the separable version of the vertical edge detection filter presented [at the beginning of this exercise](#-1.-Edge-detection-(3-points)).

In [72]:
%use javascript
%put mask_x mask_y
// // change the masks to the correct value 
var mask_x = [0, 0, 0];
var mask_y = [0, 0, 0];

// BEGIN SOLUTION
var mask_x = [-1, 0, 1];
var mask_y = [1, 2, 1];
// END SOLUTION

// convert the masks to Image objects
mask_x_img = new Image([mask_x]);
mask_y_img = new Image([mask_y]);

IPLabImageAccess { image: [ [ 1, 2, 1 ] ], nx: 3, ny: 1 }

In [73]:
%use javascript
// again let's do a sanity check with an impulse image
var impulse = new Image([[0, 0, 0], [0, 1, 0], [0, 0, 0]]);
// declare correct 2D mask
var mask = new Image([[-1, 0, 1],[-2, 0, 2],[-1, 0, 1]]);
// the result of a convolution with an impulse image should be the mask itself
console.log('Convolving by an impulse should return the unchanged mask.\nMasks:\t', mask.toArray() )
console.log('Result: ', filterSeparable(impulse, mask_x_img, mask_y_img).toArray());
if(filterSeparable(impulse, mask_x_img, mask_y_img).imageCompare(mask) == false){
    throw new Error('Sorry, the separable filter is not yet correct.');
}
console.log('Nice, the separable filter seems to be correct!');
//### BEGIN HIDDEN TESTS
// check that the column filtering was implemented correctly
var mask_x_correct = new Image([[-1, 0, 1]]);
var mask_y_correct = new Image([[1, 2, 1]]);

if(filterSeparable(new Image(bikesgray), mask_x_correct, mask_y_correct).imageCompare(filterNonSeparableCorrect(new Image(bikesgray), mask)) == false){
    throw new Error('the separable filter is not correct.');
}
//### END HIDDEN TESTS

Convolving by an impulse should return the unchanged mask.
Masks:	 [ [ -1, 0, 1 ], [ -2, 0, 2 ], [ -1, 0, 1 ] ]
Result:  [ [ -1, 0, 1 ], [ -2, 0, 2 ], [ -1, 0, 1 ] ]
Nice, the separable filter seems to be correct!


### Apply the method on the same images. 

Run the next cell to apply the method to the same three images we have been using before.

In [12]:
%use javascript
%put vedge_bike_s vedge_camera_s vedge_spots_s

// Apply method on previously defined Images
var vedge_bike_s = filterSeparable(new Image(bikesgray), mask_x_img, mask_y_img).toArray();
var vedge_spots_s = filterSeparable(new Image(spots), mask_x_img, mask_y_img).toArray();
var vedge_camera_s = filterSeparable(new Image(camera), mask_x_img, mask_y_img).toArray();
//### BEGIN HIDDEN TESTS
// check that they provide the correct masks
var mask_x_correct = new Image([[-1, 0, 1]]);
var mask_y_correct = new Image([[1, 2, 1]]);

if(mask_x_correct.imageCompare(mask_x_img) == false || mask_y_correct.imageCompare(mask_y_img) == false){
    console.log(mask_x_img.toArray(), mask_y_img.toArray())
    throw new Error('The provided masks are not correct');
}
//### END HIDDEN TESTS

Now run the next one to visualize the results.

In [13]:
%use sos
# Declare the image list
image_list_sep = [vedge_bike_s, vedge_camera_s, vedge_spots_s]
title_list = ['Bike - Vertical edges', 'Camera - Vertical edges', 'Spots - Vertical edges']

# Make sure that the images are numpy arrays and not lists
for i, image in enumerate(image_list_sep):
    image_list_sep[i] = np.array(image)

plt.close('all')
vedge_s_viewer = viewer(image_list_sep, title = title_list, widgets=True)

## <a class="anchor"></a> 1.C. Comparison: Separable vs Non-separable

Did you notice any difference between the separable and non-separable versions? Did you expect any difference? 

Explore the two histograms, zoom into different areas of the three images, and see if they are truly equal. There are several tests you can do to measure the equality of both implementations. You can even take the difference between the two images and visualize it (`image1 - image2`), or you can use the different methods provided by NumPy to compare arrays (see [numpy.testing.assert_array_equal](https://numpy.org/doc/stable/reference/generated/numpy.testing.assert_array_equal.html) and [numpy.testing.assert_array_almost_equal](https://numpy.org/doc/stable/reference/generated/numpy.testing.assert_array_almost_equal.html))  Use the next blank cell to do any necessary exploration. 

In [14]:
%use sos

### BEGIN SOLUTION
image_list_diff = []
for i in range(len(image_list_nons)):
    image_list_diff.append(image_list_nons[i] - image_list_sep[i])
    
plt.close('all')
diff_viewer = viewer(image_list_diff, title = title_list, widgets=True)
### END SOLUTION

## <a class="anchor"></a> 1.D. Edge detection in Python

In Python, the two functions `filterNonSeparate` and `filterSeparate` you implemented above are provided by the [`scipy.ndimage`](https://docs.scipy.org/doc/scipy/reference/ndimage.html) library. They are called [`convolve`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.ndimage.convolve.html) and [`convolve1d`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.ndimage.convolve1d.html), respectively. You can look at their documentation by clicking on their names.

**Example:**

To apply a non-separable filter with a 2D mask `mask` to the image `img` we use:
```python
filtered_img = ndi.convolve(img, mask, output=np.int32)
```
To apply a separable filter with two 1D masks `mask_x` and `mask_y` to the image `img` we use:
```python
# filter the columns
filtered_img = ndi.convolve1d(img, mask_y, axis=0, output=np.int32)
# filter the rows
filtered_img = ndi.convolve1d(filtered_img, mask_x, axis=1, output=np.int32)
```

<div class="alert alert-info">

**Note:** We specify `output=np.int32` in the examples because the default type of the result is the same as that of the input image. In the case of `bikesgray` this would imply `np.uint8` (8-bit unsigned integer). However, we know that the gradient can take large positive and negative values, so an unsigned integer is not the right type. Because we know that the result will still be an integer because the mask only contains integers, we choose `np.int32` (32-bit signed integers). Reflect on which output type you would need to select to process `spots` by exploring its values and histogram using the viewer at the beginning of the lab. Make sure to re-run it if it does not respond, only one viewer is active at a time!
</div>

Let's compare the Python implementations to the JavaScript implementations to make sure that we get the same result. Run the cell below to compare the different versions and review your previous code if necessary. If you did everything correctly in Sections [1.A.](#-1.A.-Non-separable-version-(1-point)) and [1.B.](#-1.B.-Separable-version-(2-points)), all four images should look exactly the same!

In [15]:
%use sos 
# Define the 2D mask
mask = np.array([[-1, 0, 1],[-2, 0, 2],[-1, 0, 1]])
# Apply non-separable Filter
vedge_ndi = ndi.convolve(bikesgray, mask, output=np.int32)
# Apply separable Filter - we use mask_x and mask_y you defined in the exercise before
# filter the columns
vedge_ndi_s = ndi.convolve1d(bikesgray, mask_y, axis=0, output=np.int32)
# filter the rows
vedge_ndi_s = ndi.convolve1d(vedge_ndi_s, mask_x, axis=1, output=np.int32)
# Visualize
plt.close('all')
vedge_viewer = viewer([np.array(vedge_bike_nons), vedge_ndi, np.array(vedge_bike_s), vedge_ndi_s], title=['JS non-separable', 'Python non-separable', 'JS separable', 'Python separable'], subplots=(2,2))

Button(description='Show Widgets', style=ButtonStyle())

# <a class="anchor"></a> 2. Implementation and classification of digital filters (3 points)

In this exercise, we will look at 6 different digital filters. Your job, **for 3 points**, is to implement each one of the filters in Python, apply them to an image, and classify their visual effect.

Note that you can define a 2D mask in Python with 
```python
np.array([[ 1,  2,  3,  4,  5], 
          [ 6,  7,  8,  9, 10], 
          [11, 12, 13, 14, 15],
          [16, 17, 18, 19, 20],
          [21, 22, 23, 24, 25]])
```
and define a 1D mask in Python with
```python
np.array([1, 2, 3, 4, 5])
```

<div class="alert alert-danger">

**Important:** If the 2D mask is separable, implement it as a separable filter using `ndi.convolve1d` with the appropriate 1D masks. Otherwise, use `ndi.convolve`. Review Section [1.D.](#-1.D.-Edge-detection-in-Python) for an example. 
</div>

## <a class="anchor"></a> 2.A. Mask A
In the cell below, **for 0.25 points**, implement the function `filter_A`, which convolves the input image with the filter

$$A =
\begin{bmatrix}
    0 & 0 & 1 & 0 & 0 \\
    0 & 1 & 2 & 1 & 0 \\
    1 & 2 & -16 & 2 & 1 \\
    0 & 1 & 2 & 1 & 0 \\
    0 & 0 & 1 & 0 & 0 \\
\end{bmatrix}\,.$$

Remember to check if the filter is separable, and in that case, implement it in that way.

In [16]:
%use sos

# Function that filters img with the mask A
def filter_A(img):
    output = img.copy()
    
    # Filter the image with mask A
    
    ### BEGIN SOLUTION
    # Define the mask
    mask = np.array([ [ 0,  0,  1,  0,  0], 
                      [ 0,  1,  2,  1,  0], 
                      [ 1,  2,-16,  2,  1],
                      [ 0,  1,  2,  1,  0],
                      [ 0,  0,  1,  0,  0]])
    
    # Perform the filtering
    output = ndi.convolve(img, mask, output=np.int32)
    
    ### END SOLUTION
    
    return output

Run the next cell to visualize the result of applying `filter_A` to the image `bikesgray`.

In [17]:
%use sos
# Apply the function
bikesgray_A = filter_A(bikesgray)
# Visualize the result
plt.close('all')
vis_A = viewer([bikesgray, bikesgray_A], title=['Original', 'Convolved with filter A'], subplots=(1,2))
### BEGIN HIDDEN TESTS
# here we test that they use ndi.convolve exactly once with the correct mask

from unittest.mock import patch

# check that ndi.convolve1d was not called
# set the ndi.convolve1d function as observed object
with patch('__main__.ndi.convolve1d') as mock_operator:
    # set the return value of the mock function
    mock_operator.return_value = np.ones(bikesgray.shape)
    # run the student function
    filter_A(bikesgray)
# check that ndi.convolve1d was not used
mock_operator.assert_not_called()

# set the ndi.convolve function as observed object
with patch('__main__.ndi.convolve') as mock_operator:
    # set the return value of the mock function
    mock_operator.return_value = np.ones(bikesgray.shape)
    # run the student function
    filter_A(bikesgray)

# check that ndi.convolve was used once
mock_operator.assert_called_once()

# Define the mask
mask_correct = np.array([ [ 0,  0,  1,  0,  0], 
                          [ 0,  1,  2,  1,  0], 
                          [ 1,  2,-16,  2,  1],
                          [ 0,  1,  2,  1,  0],
                          [ 0,  0,  1,  0,  0]])

# cycle through the calls made to the ndi.convolve function
for call_args in mock_operator.call_args_list:
    # extract the second argument passed to the function in each call and check if the correct mask was used
    if np.testing.assert_array_equal(call_args[0][1], mask_correct) == False:
        raise AssertionError("The provided mask is not correct")
        
# to be sure, check that the output is correct as well
np.testing.assert_array_almost_equal(ndi.convolve(bikesgray, mask_correct, output=np.int32), filter_A(bikesgray))
### END HIDDEN TESTS

Button(description='Show Widgets', style=ButtonStyle())

**For 0.25 points** answer the following question:<br>
Which of the following keywords best describes the visual effect of this filter?

1. isotropic blurring, or
2. vertical blurring, or
3. horizontal blurring, or
4. diagonal blurring, or
5. sharpening, or
6. vertical edge detection, or
7. horizontal edge detection, or
8. diagonal edge detection, or
9. Laplacian: a second order derivative to find edges (maxima and minima), or
10. it has not effect.

In [18]:
%use sos
# Assign your answer to this variable
answer = None
### BEGIN SOLUTION
answer = 9
### END SOLUTION

In [19]:
%use sos
# Sanity check
assert answer in list(range(1,11)), 'Answer one of 1, 2, 3, 4, 5, 6, 7, 8, 9 or 10'
### BEGIN HIDDEN TESTS
assert answer == 9
### END HIDDEN TESTS

## <a class="anchor"></a> 2.B. Mask B
In the cell below, **for 0.25 points**, implement the function `filter_B`, which convolves the input image with the filter

$$B =
\begin{bmatrix}
    0 & 1 & 2 & 1 & 0 \\
    1 & 2 & 4 & 2 & 1 \\
    2 & 4 & 8 & 4 & 2 \\
    1 & 2 & 4 & 2 & 1 \\
    0 & 1 & 2 & 1 & 0 \\
\end{bmatrix}\,.$$

Remember to check if the filter is separable, and in that case, implement it in that way.

In [20]:
%use sos

# Function that filters img with the mask B
def filter_B(img):
    output = img.copy()
    
    # Filter the image with mask B
    
    ### BEGIN SOLUTION
    # Define the mask
    mask = np.array([ [ 0,  1,  2,  1,  0], 
                      [ 1,  2,  4,  2,  1], 
                      [ 2,  4,  8,  4,  2],
                      [ 1,  2,  4,  2,  1],
                      [ 0,  1,  2,  1,  0]])
    
    # Perform the filtering
    output = ndi.convolve(img, mask, output=np.int32)
    
    ### END SOLUTION
    
    return output

Run the next cell to visualize the result of applying `filter_B` to the image `bikesgray`.

In [21]:
%use sos
# Apply the function
bikesgray_B = filter_B(bikesgray)
# Visualize the result
plt.close('all')
vis_B = viewer([bikesgray, bikesgray_B], title=['Original', 'Filtered with mask B'], subplots=(1,2))
### BEGIN HIDDEN TESTS
# here we test that they use ndi.convolve exactly once with the correct mask

from unittest.mock import patch

# check that ndi.convolve1d was not called
# set the ndi.convolve1d function as observed object
with patch('__main__.ndi.convolve1d') as mock_operator:
    # set the return value of the mock function
    mock_operator.return_value = np.ones(bikesgray.shape)
    # run the student function
    filter_B(bikesgray)
# check that ndi.convolve1d was not used
mock_operator.assert_not_called()

# set the ndi.convolve function as observed object
with patch('__main__.ndi.convolve') as mock_operator:
    # set the return value of the mock function
    mock_operator.return_value = np.ones(bikesgray.shape)
    # run the student function
    filter_B(bikesgray)

# check that ndi.convolve was used once
mock_operator.assert_called_once()
    
# Define the mask
mask_correct = np.array([ [ 0,  1,  2,  1,  0], 
                          [ 1,  2,  4,  2,  1], 
                          [ 2,  4,  8,  4,  2],
                          [ 1,  2,  4,  2,  1],
                          [ 0,  1,  2,  1,  0]])
    
# cycle through the calls made to the ndi.convolve function
for call_args in mock_operator.call_args_list:
    # extract the second argument passed to the function in each call and check if the correct mask was used
    np.testing.assert_array_equal(call_args[0][1], mask_correct)
        
# to be sure, check that the output is correct as well
np.testing.assert_array_almost_equal(ndi.convolve(bikesgray, mask_correct, output=np.int32), filter_B(bikesgray))
### END HIDDEN TESTS

Button(description='Show Widgets', style=ButtonStyle())

**For 0.25 points** answer the following question:<br>
Which of the following keywords best describes the visual effect of this filter?

1. isotropic blurring, or
2. vertical blurring, or
3. horizontal blurring, or
4. diagonal blurring, or
5. sharpening, or
6. vertical edge detection, or
7. horizontal edge detection, or
8. diagonal edge detection, or
9. Laplacian: a second order derivative to find edges (maxima and minima), or
10. it has not effect.

In [22]:
%use sos
# Assign your answer to this variable
answer = None
### BEGIN SOLUTION
answer = 1
### END SOLUTION

In [23]:
%use sos
# Sanity check
assert answer in list(range(1,11)), 'Answer one of 1, 2, 3, 4, 5, 6, 7, 8, 9 or 10'
### BEGIN HIDDEN TESTS
assert answer == 1
### END HIDDEN TESTS

## <a class="anchor"></a> 2.C. Mask C
In the cell below, **for 0.25 points**, implement the function `filter_C`, which convolves the input image with the filter

$$C =
\begin{bmatrix}
    -1 & -1 & -1 & -1 & -1 \\
    -2 & -2 & -2 & -2 & -2 \\
    0 & 0 & 0 & 0 & 0 \\
    2 & 2 & 2 & 2 & 2 \\
    1 & 1 & 1 & 1 & 1
\end{bmatrix}\,.$$

Remember to check if the filter is separable, and in that case, implement it in that way.

In [37]:
%use sos

# Function that filters img with the mask C
def filter_C(img):
    output = img.copy()
    
    # Filter the image with mask C
    
    ### BEGIN SOLUTION
    # Define the mask
    mask_x = np.array([1,  1,  1,  1,  1])
    mask_y = np.array([-1,  -2,  0,  2,  1])
    
    # Perform the filtering
    output = ndi.convolve1d(img, mask_y, axis=0, output=np.int32)
    output = ndi.convolve1d(output, mask_x, axis=1, output=np.int32)
    
    ### END SOLUTION
    
    return output

Run the next cell to visualize the result of applying `filter_C` to the image `bikesgray`.

In [38]:
%use sos
# Apply the function
bikesgray_C = filter_C(bikesgray)
# Visualize the result
plt.close('all')
vis_C = viewer([bikesgray, bikesgray_C], title=['Original', 'Filtered with mask C'], subplots=(1,2))
### BEGIN HIDDEN TESTS
# here we test that they use ndi.convolve1d exactly twice with the correct masks

from unittest.mock import patch

# check that ndi.convolve was not called
# set the ndi.convolve function as observed object
with patch('__main__.ndi.convolve') as mock_operator:
    # set the return value of the mock function
    mock_operator.return_value = np.ones(bikesgray.shape)
    # run the student function
    filter_C(bikesgray)

# check that ndi.convolve was used once
mock_operator.assert_not_called()

# set the ndi.convolve1d function as observed object
with patch('__main__.ndi.convolve1d') as mock_operator:
    # set the return value of the mock function
    mock_operator.return_value = np.ones(bikesgray.shape)
    # run the student function
    filter_C(bikesgray)
# check that ndi.convolve1d was used twice
mock_operator.assert_called()
if len(mock_operator.call_args_list) != 2:
    raise AssertionError('Separable filter was not implemented correctly')
    
# Define the masks
mask_x_correct = np.array([1,  1,  1,  1,  1])
mask_y_correct = np.array([-1,  -2,  0,  2,  1])
# Student masks
mask_x_test = None
mask_y_test = None
    
# Cycle through the calls made to the ndi.convolve function
for call_args in mock_operator.call_args_list:
    # Extract the second argument passed to the function in each call and check if the correct mask was used
    axis = None
    try:
        axis = call_args[1]['axis']
    except KeyError:
        axis = call_args[0][2]
    if axis == 0:
        # axis 0
        mask_y_test = call_args[0][1]
    elif axis == 1:
        # axis 1
        mask_x_test = call_args[0][1]

# Check the student masks
assert mask_x_correct @ mask_y_correct == mask_x_test @ mask_y_test, 'The masks are not correct.'

# to be sure, check that the output is correct as well
np.testing.assert_array_almost_equal(ndi.convolve1d(ndi.convolve1d(bikesgray, mask_y_correct, axis=0, output=np.int32), mask_x_correct, axis=1, output=np.int32), filter_C(bikesgray))
### END HIDDEN TESTS

Button(description='Show Widgets', style=ButtonStyle())

**For 0.25 points** answer the following question:<br>
Which of the following keywords best describes the visual effect of this filter?

1. isotropic blurring, or
2. vertical blurring, or
3. horizontal blurring, or
4. diagonal blurring, or
5. sharpening, or
6. vertical edge detection, or
7. horizontal edge detection, or
8. diagonal edge detection, or
9. Laplacian: a second order derivative to find edges (maxima and minima), or
10. it has not effect.

In [41]:
%use sos
# Assign your answer to this variable
answer = None
### BEGIN SOLUTION
answer = 7
### END SOLUTION

In [42]:
%use sos
# Sanity check
assert answer in list(range(1,11)), 'Answer one of 1, 2, 3, 4, 5, 6, 7, 8, 9 or 10'
### BEGIN HIDDEN TESTS
assert answer == 7
### END HIDDEN TESTS

## <a class="anchor"></a> 2.D. Mask D
In the cell below, **for 0.25 points**, implement the function `filter_D`, which convolves the input image with the filter

$$D = 
\begin{bmatrix}
-1 & -4 & -6 & -4 & -1 \\
-4 & -16 & -24 & -16 & -4 \\
-6 & -24 & 476 & -24 & -6 \\
-4 & -16 & -24 & -16 & -4 \\
-1 & -4 & -6 & -4 & -1
\end{bmatrix}\,.$$

Remember to check if the filter is separable, and in that case, implement it in that way.

In [43]:
%use sos

# Function that filters img with the mask D
def filter_D(img):
    output = img.copy()
    
    # Filter the image with mask D
    
    ### BEGIN SOLUTION
    # Define the mask
    mask = np.array([ [-1, -4, -6, -4, -1], 
                      [-4,-16,-24,-16, -4], 
                      [-6,-24,476,-24, -6],
                      [-4,-16,-24,-16, -4],
                      [-1, -4, -6, -4, -1]])
    
    # Perform the filtering
    output = ndi.convolve(img, mask, output=np.int32)
    
    ### END SOLUTION
    
    return output

Run the next cell to visualize the result of applying `filter_D` to the image `bikesgray`.

In [44]:
%use sos
# Apply the function
bikesgray_D = filter_D(bikesgray)
# Visualize the result
plt.close('all')
vis_D = viewer([bikesgray, bikesgray_D], title=['Original', 'Filtered with mask D'], subplots=(1,2))
### BEGIN HIDDEN TESTS
# here we test that they use ndi.convolve exactly once with the correct mask

from unittest.mock import patch

# check that ndi.convolve1d was not called
# set the ndi.convolve1d function as observed object
with patch('__main__.ndi.convolve1d') as mock_operator:
    # set the return value of the mock function
    mock_operator.return_value = np.ones(bikesgray.shape)
    # run the student function
    filter_D(bikesgray)
# check that ndi.convolve1d was used twice
mock_operator.assert_not_called()

# set the ndi.convolve function as observed object
with patch('__main__.ndi.convolve') as mock_operator:
    # set the return value of the mock function
    mock_operator.return_value = np.ones(bikesgray.shape)
    # run the student function
    filter_D(bikesgray)

# check that ndi.convolve was used once
mock_operator.assert_called_once()
    
# Define the mask
mask_correct = np.array([ [-1, -4, -6, -4, -1], 
                          [-4,-16,-24,-16, -4], 
                          [-6,-24,476,-24, -6],
                          [-4,-16,-24,-16, -4],
                          [-1, -4, -6, -4, -1]])
    
# cycle through the calls made to the ndi.convolve function
for call_args in mock_operator.call_args_list:
    # extract the second argument passed to the function in each call and check if the correct mask was used
    np.testing.assert_array_equal(call_args[0][1], mask_correct)
        
# to be sure, check that the output is correct as well
np.testing.assert_array_almost_equal(ndi.convolve(bikesgray, mask_correct, output=np.int32), filter_D(bikesgray))
### END HIDDEN TESTS

Button(description='Show Widgets', style=ButtonStyle())

**For 0.25 points** answer the following question:<br>
Which of the following keywords best describes the visual effect of this filter?

1. isotropic blurring, or
2. vertical blurring, or
3. horizontal blurring, or
4. diagonal blurring, or
5. sharpening, or
6. vertical edge detection, or
7. horizontal edge detection, or
8. diagonal edge detection, or
9. Laplacian: a second order derivative to find edges (maxima and minima), or
10. it has not effect.

In [45]:
%use sos
# Assign your answer to this variable
answer = None
### BEGIN SOLUTION
answer = 5
### END SOLUTION

In [46]:
%use sos
# Sanity check
assert answer in list(range(1,11)), 'Answer one of 1, 2, 3, 4, 5, 6, 7, 8, 9 or 10'
### BEGIN HIDDEN TESTS
assert answer == 5
### END HIDDEN TESTS

## <a class="anchor"></a> 2.E. Mask E
In the cell below, **for 0.25 points**, implement the function `filter_E`, which convolves the input image with the filter 

$$E = 
\begin{bmatrix}
    0 & 0 & 0 & 0 & 0 \\
    0 & 0 & 0 & 0 & 0 \\
    0 & 0 & 1 & 0 & 0 \\
    0 & 0 & 0 & 0 & 0 \\
    0 & 0 & 0 & 0 & 0 \\
\end{bmatrix}\,.$$

Remember to check if the filter is separable, and in that case, implement it in that way.

In [51]:
%use sos

# Function that filters img with the mask E
def filter_E(img):
    output = img.copy()
    
    # Filter the image with mask E
    
    ### BEGIN SOLUTION
    # Define the mask
    mask_x = np.array([0,  0,  1,  0,  0])
    mask_y = np.array([0,  0,  1,  0,  0])
    
    # Perform the filtering
    output = ndi.convolve1d(img, mask_y, axis=0, output=np.int32)
    output = ndi.convolve1d(output, mask_x, axis=1, output=np.int32)
    
    ### END SOLUTION
    
    return output

Run the next cell to visualize the result of applying `filter_E` to the image `bikesgray`.

In [52]:
%use sos
# Apply the function
bikesgray_E = filter_E(bikesgray)
# Visualize the result
plt.close('all')
vis_E = viewer([bikesgray, bikesgray_E], title=['Original', 'Filtered with mask E'], subplots=(1,2))
### BEGIN HIDDEN TESTS
# here we test that they use ndi.convolve1d exactly twice with the correct masks

from unittest.mock import patch

# check that ndi.convolve was not called
# set the ndi.convolve function as observed object
with patch('__main__.ndi.convolve') as mock_operator:
    # set the return value of the mock function
    mock_operator.return_value = np.ones(bikesgray.shape)
    # run the student function
    filter_E(bikesgray)

# check that ndi.convolve was used once
mock_operator.assert_not_called()

# set the ndi.convolve1d function as observed object
with patch('__main__.ndi.convolve1d') as mock_operator:
    # set the return value of the mock function
    mock_operator.return_value = np.ones(bikesgray.shape)
    # run the student function
    filter_E(bikesgray)
# check that ndi.convolve1d was used twice
mock_operator.assert_called()
if len(mock_operator.call_args_list) != 2:
    raise AssertionError('Separable filter was not implemented correctly')
    
# Define the masks
mask_x_correct = np.array([0,  0,  1,  0,  0])
mask_y_correct = np.array([0,  0,  1,  0,  0])
# Student masks
mask_x_test = None
mask_y_test = None
    
# cycle through the calls made to the ndi.convolve function
for call_args in mock_operator.call_args_list:
    # extract the second argument passed to the function in each call and check if the correct mask was used
    axis = None
    try:
        axis = call_args[1]['axis']
    except KeyError:
        axis = call_args[0][2]
    if axis == 0:
        # axis 0
        mask_y_test = call_args[0][1]
    elif axis == 1:
        # axis 1
        mask_x_test = call_args[0][1]

# Check the student masks
assert mask_x_correct @ mask_y_correct == mask_x_test @ mask_y_test, 'The masks are not correct.'

# to be sure, check that the output is correct as well
np.testing.assert_array_almost_equal(ndi.convolve1d(ndi.convolve1d(bikesgray, mask_y_correct, axis=0, output=np.int32), mask_x_correct, axis=1, output=np.int32), filter_E(bikesgray))
### END HIDDEN TESTS

Button(description='Show Widgets', style=ButtonStyle())

**For 0.25 points** answer the following question:<br>
Which of the following keywords best describes the visual effect of this filter?

1. isotropic blurring, or
2. vertical blurring, or
3. horizontal blurring, or
4. diagonal blurring, or
5. sharpening, or
6. vertical edge detection, or
7. horizontal edge detection, or
8. diagonal edge detection, or
9. Laplacian: a second order derivative to find edges (maxima and minima), or
10. it has not effect.

In [53]:
%use sos
# Assign your answer to this variable
answer = None
### BEGIN SOLUTION
answer = 10
### END SOLUTION

In [54]:
%use sos
# Sanity check
assert answer in list(range(1,11)), 'Answer one of 1, 2, 3, 4, 5, 6, 7, 8, 9 or 10'
### BEGIN HIDDEN TESTS
assert answer == 10
### END HIDDEN TESTS

## <a class="anchor"></a> 2.F. Mask F
In the cell below, **for 0.25 points**, implement the function `filter_F`, which convolves the input image with the filter 

$$F = 
\begin{bmatrix}
    0 & -1 & -1 & 0 & 0 \\
    -1 & -1 & 0 & 0 & 0 \\
    -1 & 0 & 0 & 0 & 1 \\
    0 & 0 & 0 & 1 & 1 \\
    0 & 0 & 1 & 1 & 0 \\
\end{bmatrix}\,.$$

Remember to check if the filter is separable, and in that case, implement it in that way.

In [55]:
%use sos

# Function that filters img with the mask F
def filter_F(img):
    output = img.copy()
    
    # Filter the image with mask F
    
    ### BEGIN SOLUTION
    # Define the mask
    mask = np.array([ [  0, -1, -1,  0,  0], 
                      [ -1, -1, -1,  0,  0], 
                      [ -1,  0,  0,  0,  1],
                      [  0,  0,  0,  1,  1],
                      [  0,  0,  1,  1,  0]])
    
    # Perform the filtering
    output = ndi.convolve(img, mask, output=np.int32)
    
    ### END SOLUTION
    
    return output

Run the next cell to visualize the result of applying `filter_F` to the image `bikesgray`.

In [56]:
%use sos
# Apply the function
bikesgray_F = filter_F(bikesgray)
# Visualize the result
plt.close('all')
vis_F = viewer([bikesgray, bikesgray_F], title=['Original', 'Filtered with mask F'], subplots=(1,2))
### BEGIN HIDDEN TESTS
# here we test that they use ndi.convolve exactly once with the correct mask

from unittest.mock import patch

# check that ndi.convolve1d was not called
# set the ndi.convolve1d function as observed object
with patch('__main__.ndi.convolve1d') as mock_operator:
    # set the return value of the mock function
    mock_operator.return_value = np.ones(bikesgray.shape)
    # run the student function
    filter_F(bikesgray)
# check that ndi.convolve1d was used twice
mock_operator.assert_not_called()

# set the ndi.convolve function as observed object
with patch('__main__.ndi.convolve') as mock_operator:
    # set the return value of the mock function
    mock_operator.return_value = np.ones(bikesgray.shape)
    # run the student function
    filter_F(bikesgray)

# check that ndi.convolve was used once
mock_operator.assert_called_once()
    
# Define the mask
mask_correct = np.array([ [  0, -1, -1,  0,  0], 
                          [ -1, -1, -1,  0,  0], 
                          [ -1,  0,  0,  0,  1],
                          [  0,  0,  0,  1,  1],
                          [  0,  0,  1,  1,  0]])
    
# cycle through the calls made to the ndi.convolve function
for call_args in mock_operator.call_args_list:
    # extract the second argument passed to the function in each call and check if the correct mask was used
    np.testing.assert_array_equal(call_args[0][1], mask_correct)
        
# to be sure, check that the output is correct as well
np.testing.assert_array_almost_equal(ndi.convolve(bikesgray, mask_correct, output=np.int32), filter_F(bikesgray))
### END HIDDEN TESTS

Button(description='Show Widgets', style=ButtonStyle())

**For 0.25 points** answer the following question:<br>
Which of the following keywords best describes the visual effect of this filter?

1. isotropic blurring, or
2. vertical blurring, or
3. horizontal blurring, or
4. diagonal blurring, or
5. sharpening, or
6. vertical edge detection, or
7. horizontal edge detection, or
8. diagonal edge detection, or
9. Laplacian: a second order derivative to find edges (maxima and minima), or
10. it has not effect.

In [57]:
%use sos
# Assign your answer to this variable
answer = None
### BEGIN SOLUTION
answer = 8
### END SOLUTION

In [58]:
%use sos
# Sanity check
assert answer in list(range(1,11)), 'Answer one of 1, 2, 3, 4, 5, 6, 7, 8, 9 or 10'
### BEGIN HIDDEN TESTS
assert answer == 8
### END HIDDEN TESTS

## <a class="anchor"></a> 3. Gaussian Filter (4 Points)

In this exercise we will implement a very popular low-pass filter, the Gaussian filter. The Gaussian filter is used for noise reduction, realistic blur, and many other tasks. 

We propose to implement the 2D Gaussian filter with impulse response $h_{\sigma}[m,n]$, where $\sigma$ is the standard deviation of an isotropic 2D Gaussian and controls the smoothing strength. This impulse response discretizes the 2D Gaussian function $h_\sigma(x,y)$ between $[-\lceil3\sigma\rceil,\lceil 3\sigma\rceil]$ in $x$ and $y$. You will choose the size of the filter to be $N = 2\lceil 3\sigma \rceil+1$ (hence, $N$ is always odd), and you will ensure your impulse response adds up to $1$ using the appropriate normalization. Here, $\lceil x \rceil$ refers to the ceiling function, i.e., the smallest integer larger than $x$. 

Remember that the expression of an [isotropic 2D Gaussian](https://en.wikipedia.org/wiki/Multivariate_normal_distribution) is

$$h_\sigma(x,y) = \frac{1}{2\pi\sigma^2}\exp\left(-\frac{x^2 + y^2}{2\sigma^2}\right) = \frac{1}{\sqrt{2\pi}\sigma}\exp\left(-\frac{x^2}{2\sigma^2}\right) \frac{1}{\sqrt{2\pi}\sigma}\exp\left(-\frac{ y^2}{2\sigma^2}\right) \,.$$

## <a class="anchor"></a> 3.A. Implementation of a 2D Gaussian filter (4 points)

For **3 points, implement the method `gaussian(img, sigma)`** that convolves an image with a Gaussian filter using a separable implementation. If you wish, you can take advantage of the method `filter1D` defined in Section [1.B.](#-1.B.-Separable-version-(2-points)).

<div class="alert alert-info">
    <b>Hint: </b> Remember that you can use <a href="https://developer.mozilla.org/en-US/docs/Web/JavaScript/Reference/Global_Objects/Math">the <code>Math</code> library</a> to access different mathematical functions (<code>Math.ceil()</code>, <code>Math.floor()</code>, <code>Math.exp()</code>, <code>Math.PI</code>, <code>Math.sqrt()</code>).
</div>

<div class="alert alert-info">

**Remember:** The first argument to the `Image` constructor is the **height** and the second is the **width**: <br>`new Image(height, width)` or `new Image([height, width])`
</div>

In [74]:
%use javascript
// function that performs a gaussian filter with sigma on img
function gaussian(img, sigma){
    // declare output variable
    var output = new Image(img.shape());
    
    // BEGIN SOLUTION 
    // mask size
    var N = 2* Math.ceil(3*sigma) + 1;
    // define mask and fill it's values
    var mask =  new Image(1, N);   
    var norm = 0;
    for(var x = 0; x < N; x++){  
        mask.setPixel(x, 0, 1 / (Math.sqrt(2*Math.PI)*sigma) * Math.exp( - ( Math.pow( x-Math.floor(N/2), 2 ) / ( 2*Math.pow(sigma,2) ) ) ) )
        norm += mask.getPixel(x,0)
    } 
    for(var x = 0; x < N; x++){
        mask.setPixel(x,0,mask.getPixel(x,0)/norm)
    }
    // iterate through every column 
    for(var y = 0; y < img.ny; y++){
        // extract row
        var row = img.getRow(y);
        // apply filter
        var new_row = filter1D(row, mask)
        // set column in output variable
        output.putRow(y, new_row);
    }
    // iterate through every row
    for(var x = 0; x < img.nx; x++){
        // extract column
        var column = output.getColumn(x);
        // apply filter
        var new_column = filter1D(column, mask) 
        // set column in output variable
        output.putColumn(x, new_column);
        // END SOLUTION
        
    }    
    return output
}

We have designed a quick test for you to evaluate your method, applying it to a $3 \times 3$ impulse image. Run the following cell and check that your output has all the desired properties of a Gaussian.

In [75]:
%use javascript
%put impulse_gaussian
// define the impulse image
var impulse = new Image([[0, 0, 0], [0, 1, 0], [0, 0, 0]])

// apply filter to previously defined impulse
var impulse_gaussian = gaussian(impulse, 0.5).toArray()

// look at result, verify that it has the properties of a gaussian
console.log(impulse_gaussian)

// this assertion checks proper behaviour: that the center is the maximum, and that two pixels in equivalent positions have the same values.
if(impulse_gaussian[1][1] < impulse_gaussian[2][1] || impulse_gaussian[0][1] !== impulse_gaussian[2][1]){
    throw new Error("Implementation not correct");
}else{console.log('Well done! Your implementation seems to be correct.')}

// check normalization
sum = 0
for(var x = 0; x < impulse_gaussian.length; x++){
    for (var y =0; y < impulse_gaussian.length; y++){
        sum += impulse_gaussian[y][x]
    }
}
if (Math.abs(sum - 1) > 1e-5){
    throw new Error("Normalization not correct");
}else{console.log("Well done! The output sums up to approximately 1.")}
    
// BEGIN HIDDEN TESTS
// apply filter with sigma = 0.5 (last assertions will work with any sigma)
var impulse_gaussian_ta = gaussian(impulse, 0.5).toArray()

// check that the filter is correctly implemented
if(Math.abs( impulse_gaussian_ta[1][1]/impulse_gaussian_ta[2][1] - 7.37078574771476 ) > 1e-10 ){
    throw new Error("Implementation not correct");
}
// END HIDDEN TESTS

[
  [ 0.011388013762064005, 0.0839386095322009, 0.011388013762064005 ],
  [ 0.08393860953220092, 0.6186935068229408, 0.08393860953220092 ],
  [ 0.011388013762064005, 0.0839386095322009, 0.011388013762064005 ]
]
Well done! Your implementation seems to be correct.
Well done! The output sums up to approximately 1.


Now that you have tested your Gaussian filter, apply it to the image `bikesgray`. Use different values of $\sigma$. Look at the evolution of the mean and the standard deviation (you can get them from the statistics box in the `IPLabViewer` class, or you can use the functions `np.mean` and `np.std`). Then, answer the two multiple choice questions.

Run and modify the two following cells to apply Gaussian filters with different $\sigma$ values to `bikesgray` and view the result. 

In [76]:
%use javascript
%put bikesgray_gaussian1 bikesgray_gaussian5

// apply filter to Image object. To try different sigma values, change the variables or declare more. 
var bikesgray_gaussian1 = gaussian(new Image(bikesgray), 1).toArray()
var bikesgray_gaussian5 = gaussian(new Image(bikesgray), 5).toArray()

In [77]:
%use sos

# Declare parameters for IPLabViewer. If you want to visualize more sigma values, update the previous cell and these lists accordingly
image_list_blur = [bikesgray, bikesgray_gaussian1, bikesgray_gaussian5]
title_list_blur = ['Original', 'Sigma: 1', 'Sigma: 5']
# Make sure that the object used is a numpy array
for i in range(len(image_list_blur)):
    image_list_blur[i] = np.array(image_list_blur[i])

# To allow a direct comparison of the images.
plt.close('all')
blurred_bikesgray_viewer = viewer(image_list_blur, title = title_list_blur, hist = True)

### Multiple Choice Question

After modifying the two cells above and visualizing the results, answer the next two questions (worth **0.5 points** each).

* Q1: With a higher $\sigma$, what happens to the mean and the standard deviation? How would you expect the Fourier transform to change?
    1. The mean and the standard deviation are lower. The Fourier transform will show lower values for higher frequencies.
    2. The mean and the standard deviation are higher. The Fourier transform will show lower values for higher frequencies.
    3. The mean and the standard deviation do not change. The Fourier transform will show higher values for higher frequencies.
    4. The mean is higher and the standard deviation does not change. The Fourier transform will show lower values for higher frequencies.
    5. The mean does not change and the standard deviation is lower. The Fourier transform will show lower values for higher frequencies.


* Q2: What will be the output image when $\sigma\rightarrow \infty$? What type of filter would that be?
    1. An image equal to the original. It would be an all-pass filter.
    2. A constant image. It would be a high-pass filter.
    3. A 2D Gaussian. It would be a band-pass filter.
    4. A constant image. It would be a low-pass filter.

Modify the variables `answer_one` and `answer_two` in the next cell to match your choices. The second and third cells are for you to make sure that your answer is valid (they should not raise any error).

In [78]:
%use sos
# Modify these variables
answer_one = None
answer_two = None
### BEGIN SOLUTION
answer_one = 5
answer_two = 4
### END SOLUTION

In [79]:
%use sos
# Sanity test
assert answer_one in [1, 2, 3, 4, 5], 'Answer one of 1, 2, 3, 4 or 5'
### BEGIN HIDDEN TESTS
assert answer_one == 5
### END HIDDEN TESTS

In [80]:
%use sos
# Sanity test
assert answer_two in [1, 2, 3, 4], 'Answer one of 1, 2, 3 or 4'
### BEGIN HIDDEN TESTS
assert answer_two == 4
### END HIDDEN TESTS

## <a class="anchor"></a> 3.B. Gaussian filter in Python

There are several implementations of Gaussian filters in Python. We will use the `scikit-image` implementation (see its documentation [here](https://scikit-image.org/docs/dev/api/skimage.filters.html#skimage.filters.gaussian)). The basic syntax is the following:

```python
output = skimage.filters.gaussian(input, sigma, mode, truncate, preserve_range = True)
```

The parameters are:
* `input` (numpy array): Original image
* `sigma` (float): $\sigma$ value.
* `mode` (string): Boundary conditions. As in the rest of the course, we will be using `'reflect'`.
* `truncate` (float): truncate the filter at this many standard deviations. Defaults to 4, but we will use 3 to agree with the implementation in JavaScript.
* `preserve_range` (boolean): indicates whether to convert the image to a floating point value between $0$ and $1$ (`False`), or it simply normalizes the Gaussian kernel (`True`). 

The output is the filtered image. 

In the next cells, we will give you an example on the image `bikesgray`. Furthermore, we will compare it to your implementation. 

Run the next cell to get a blurred version of bikesgray using the skimage gaussian filter. We will use $\sigma = 10$.

In [81]:
%use sos

bikesgray_gaussian_skimage = skimage.filters.gaussian(bikesgray, sigma = 10 , mode = 'reflect', truncate = 3, preserve_range = True)

Now, we will compare it to your implementation in JavaScript to make sure that they are equivalent (up to errors on the order of $10^{-14}$). For this, we call the `gaussian` method you implemented with the image bikesgray, also for $\sigma = 10$. Run the next cell to get the variable `bikesgray_gaussian10`.

In [82]:
%use javascript
%put bikesgray_gaussian10

// apply filter to Image object
var bikesgray_gaussian10 = gaussian(new Image(bikesgray), 10).toArray()

Now we will look at each of them and at their differences.

In [83]:
%use sos

# Make sure that the one imported from JavaScript is a numpy array
bikesgray_gaussian10 = np.array(bikesgray_gaussian10)

# Declare parameters of viewer
image_list = [bikesgray_gaussian10, bikesgray_gaussian_skimage, np.abs(bikesgray_gaussian_skimage - bikesgray_gaussian10)]
title_list = ['JS', 'Skimage', 'Difference']

# We call the viewer with clip_range = [0, 1] to compare the difference with respect to the originals
plt.close('all')
skimage_gaussian_viewer = viewer(image_list, title = title_list, subplots = [1, 3], widgets = True)

### BEGIN HIDDEN TESTS
np.testing.assert_array_almost_equal(bikesgray_gaussian10, bikesgray_gaussian_skimage, decimal = 10)
### END HIDDEN TESTS

# <a class="anchor"></a> 4. Application: Spot detector (5 points)

Now that you have implemented several filters and you master the concepts behind digital filtering, we are going to lead you in a real application of Gaussian filtering. You are going to implement a *spot detector*, based on the Difference of Gaussians (DoG) filter. From now on, you will only be using Python libraries.

A good algorithm to detect spots is to compute the local maximum on the output of the DoG filter (Difference of Gaussians), which is an approximation of the Laplacian-of-Gaussian (LoG) filter explained on _page 5-35_ of the course notes.

In the following two exercises we'll be using the image `spots`. Run the next cell to visualize it.

In [84]:
%use sos
plt.close('all')
spots_vis = viewer(spots)

Button(description='Show Widgets', style=ButtonStyle())

## <a class="anchor"></a> 4.A. Difference of Gaussians (2 points)

The DoG is constructed from the subtraction of two Gaussian functions, i.e., $\mathrm{DoG}(x) = h_{\sigma_{1}}(x) - h_{\sigma_2}(x)$. It is usually parametrised only by $\sigma_1$, and $\sigma_2$ is chosen as $\sigma_2 = \sqrt{2}\sigma_1$. Experiment with the value of $\sigma_1$ in the next cell to see the kind of profile generated by this filter in 1D. 

In [85]:
import scipy
# Choose sigmas
sigma_1 = 1; sigma_2 = np.sqrt(2)*sigma_1
# Generate axis to plot
x = np.arange(-3*sigma_2, (3+6./100)*sigma_2, 6*sigma_2/100)
# Plot both Gaussians 
plt.close("all"); plt.figure(f"Difference of Gaussians filter in 1D - SCIPER: {uid}", figsize = [10,4])
plt.plot(x, scipy.stats.norm(scale=sigma_1).pdf(x), x, scipy.stats.norm(scale=sigma_2).pdf(x),
         x, scipy.stats.norm(scale=sigma_1).pdf(x) - scipy.stats.norm(scale=sigma_2).pdf(x));
plt.xlabel(r"$x$"); plt.legend([r"$h_{\sigma_1}(x)$", r"$h_{\sigma_2}(x)$", r"$\mathrm{DoG}_{\sigma_1}(x)$"]);
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Pan', 'Pan axes with left…

For **1 point**, modify the next cell and write the function `dog(image, sigma1)`. This method should implement the DoG filter by combining the result of two Gaussian filters. We recommend you to use the `scikit-image` implementation that we used in Section [3.B.](#-3.B.-Gaussian-filter-in-Python) , but feel free to use any library you want. Make sure you use the `preserve_range = True` option (or equivalent), to maintain the values of the filtered images with respect to each other. Normalize the output of your function so that it expands the range $[0,1]$.

In [86]:
%use sos

def dog(image, sigma_1):
    output = np.copy(image)
    ### BEGIN SOLUTION
    sigma_2 = np.sqrt(2)*sigma_1
    filt1 = skimage.filters.gaussian(image, sigma = sigma_1 , mode = 'reflect', truncate = 3, preserve_range = True)
    filt2 = skimage.filters.gaussian(image, sigma = sigma_2 , mode = 'reflect', truncate = 3, preserve_range = True)
    output = filt1 - filt2
    # normalize output
    output = (output - np.amin(output)) /(np.max(output) - np.min(output))
    ### END SOLUTION
    return output

err_message = "Remember to normalize the output so that it expands the range [0,1]."
assert dog( spots, 1 ).max() == 1, err_message
assert dog( spots, 1 ).min() == 0, err_message

In the next cell, you will visualize the results of your function for different $\sigma$ values. We will declare a slider with values in the range $[0-10]$, a button and an activation function to get the value of the slider and apply your function to an input image. 

First, run the next cell to declare these widgets and the function.

In [87]:
%use sos
# Define sliders and button
sigma_slider = widgets.FloatSlider(value=1, min=0.5, max=10.0, step=0.5, description='\u03c3\u2081:')
button = widgets.Button(description = 'Apply DoG')
# Define callback function
def button_dog(image):
    sigma = sigma_slider.value
    image = dog(image, sigma)
    return image

Now run the next cell to visualize the results. Go to the menu *Extra Widgets*, where you can find the slider. You will apply it to the image `spots`. If you want to experiment with the filter using other values of $\sigma_1$, you can modify the values `min`, `max` and `step` in the cell above. 

In [88]:
%use sos
# Visualize the dog spots
plt.close("all")
dog_viewer = viewer(spots, title = "DoG Spots", new_widgets = [sigma_slider, button], callbacks = [button_dog], widgets = True, normalize = True)

### BEGIN HIDDEN TESTS
def dog_ta(image, sigma1):
    output = np.copy(image)
    sigma2 = np.sqrt(2)*sigma1
    filt1 = skimage.filters.gaussian(image, sigma = sigma1 , mode = 'reflect', truncate = 3, preserve_range = True)
    filt2 = skimage.filters.gaussian(image, sigma = sigma2 , mode = 'reflect', truncate = 3, preserve_range = True)
    output = filt1 - filt2
    # normalize output
    output = (output - np.amin(output)) /(np.amax(output) - np.amin(output))
    return output

dog_spots_ta = dog_ta(spots, 1)
dog_spots = dog(spots, 1)

# Test that arrays are equal to a certain extent
np.testing.assert_array_almost_equal(dog_spots_ta, dog_spots, decimal = 5)
### END HIDDEN TESTS

### Multiple choice question (1 point)

* Q1: What type of filter is the DoG?

    1. Low-pass  
    2. Band-pass  
    3. High-pass  


* Q2: Which $\sigma$ would you choose to highlight the spots?

    1. 1.5  
    2. 5  
    3. 10 


Modify the variables `answer_one` and `answer_two` in the next cell to your choices. Use the subsequent two cells to verify that your answers are valid.

In [89]:
%use sos
# Modify these variables
answer_one = None
answer_two = None
### BEGIN SOLUTION
answer_one = 2
answer_two = 1
### END SOLUTION

In [90]:
%use sos
# Sanity check
assert answer_one in [1, 2, 3], 'Choose one of 1, 2 or 3.'
### BEGIN HIDDEN TESTS
assert answer_one == 2
### END HIDDEN TESTS

In [91]:
%use sos
# Sanity check
assert answer_two in [1, 2, 3], 'Choose one of 1, 2 or 3.'
### BEGIN HIDDEN TESTS
assert answer_two == 1
### END HIDDEN TESTS

## <a class="anchor"></a> 4.B. Local maxima (1 point)

Now you will write the function `local_max(img, T)` that returns a binary image. This function will set the pixels which are a local maximum in a $3\times 3$ neighbourhood to the maximum value of the image, and any other pixels to $0$. A local maximum is a pixel that has a value strictly greater than its 8 closest neighbors (8-connected) and is strictly greater than a threshold $T$ (relative to the maximum of the image).

<div class="alert alert-info">

**Remember:** 8-connected pixels are neighbors to every pixel that touches one of their edges or corners. <img src="images/8_connectivity.jpg" alt="8-connectivity" width="100px">
</div>

Remember that Image Processing libraries can do most of the work for you. `scikit-image` has the method `skimage.feature.peak_local_max()` ([see documentation](https://scikit-image.org/docs/0.7.0/api/skimage.feature.peak)), which allows you to specify the minimum distance between peaks and a threshold relative to the maximum value of the image. It outputs the coordinates of the corresponding pixels. Once you have the coordinates, you can use them to index an array and put put the desired value in the appropriate places. For example, if we have a NumPy array `peaks` with local maxima locations as the one returned by the aforementioned function, you can index an image as `output[peaks[:,0], peaks[:,1]] = value`.

For **1 point**, modify the next cell and define your function.

In [92]:
%use sos
# Import the module feature from skimage if you are going to use it
from skimage import feature

# Function that computes the local max in a 3x3 nbh
def local_max(img, T):
    output = np.zeros(img.shape)
    ### BEGIN SOLUTION
    coord = feature.peak_local_max(img, min_distance=1, threshold_rel = T)
    output[coord[:,0],coord[:,1]] = np.max(img)
    ### END SOLUTION
    return output

Run the next cell for a quick test on your function. In it, we test  that your image applied to `camera` with a threshold $t = 0.5$ detects exactly the four maximum points of the image, as it should. If the assertion raises no error, your function is most probably correct.

In [93]:
%use sos

assert np.count_nonzero(local_max(camera, 0.5)) == 4, 'Your function its not yet correct!'
print('Congratulations! Your function is correctly detecting the 4 spots.')
### BEGIN HIDDEN TESTS
def local_max_ta(img, T):
    output = np.zeros(img.shape)
    coord = feature.peak_local_max(img, min_distance=1, threshold_rel = T)
    coord = tuple(np.transpose(coord).tolist())
    output[coord] = np.amax(img)
    return output

camera_lmax_ta = local_max_ta(camera, 0.6)
camera_lmax = local_max(camera, 0.6)

assert np.count_nonzero(camera_lmax_ta - camera_lmax) == 0
### END HIDDEN TESTS

Congratulations! Your function is correctly detecting the 4 spots.


Now you are going to see the effect of this function through a slider on the `IPLabViewer`. Like in the exercise to visualize the effects of `dog()`, we will declare one slider for the threshold and one button to call the `local_max` method. 

Run the next cell to test it on the image `camera`. Do you think that this function would be able to detect the **6 spots** in `spots`?  Modify the next cell and look at the result of your function applied to other images!

In [94]:
%use sos

threshold_slider = widgets.FloatSlider(value=0, min=0, max=1, step=0.01, description='T:')
button = widgets.Button(description = 'Apply Local Maxima')

def button_local_max(image):
    t = threshold_slider.value
    image = local_max(image, t)
    return image

local_max_viewer = viewer(camera, title = "Local Maxima", new_widgets = [threshold_slider, button], callbacks = [button_local_max], widgets=True)

## <a class="anchor"></a> 4.C. Spot detector (2 points)

For **1 point**, implement the method `spot_detector(img, sigma, T)`, where you use your previous two functions to detect spots. In other words, apply the detection of local maxima on the output of the DoG filter.

In [95]:
%use sos

# Function that detects spots in img, using sigma and a threshold T
def spot_detector(img, sigma, T):
    ### BEGIN SOLUTION
    output = dog(img, sigma)
    output = local_max(output, T)
    ### END SOLUTION
    return output

Run the next cell for a quick test on your function.

In [96]:
%use sos

assert np.count_nonzero(spot_detector(spots, 1, 0.3)) == 6, 'Your function is not yet correct. First make sure that dog() and local_max() are.'
print('Congratulations! Your spot detector seems to be correct.')
### BEGIN HIDDEN TESTS
def spot_detector_ta(img, sigma, T):
    output = dog_ta(img, sigma)
    output = local_max_ta(output, T)
    return output

spots_spots_ta = spot_detector_ta(spots, 1, 0.3)
spots_spots = spot_detector(spots, 1, 0.3)

np.testing.assert_array_almost_equal(spots_spots_ta, spots_spots, decimal = 5)
### END HIDDEN TESTS

Congratulations! Your spot detector seems to be correct.


Now, lets apply your function to the image `spots` inside an `IPLabViewer`, using two sliders for the values of $\sigma_1$ and $T$. 

Run the following cell, and play with these values (access the sliders through the button *Extra Widgets*). Explore the results also on other images.

<div class="alert alert-info">

**Note:** Because it can be hard to see single white pixels on some screens (especially if they're dusty) we also print the number of spots that have been detected when applying the function in the viewer.
</div>

In [105]:
%use sos

# Define sliders
sigma_slider = widgets.FloatSlider(value=1, min=0.5, max=10.0, step=0.5, description="\u03c3\u2081:")
t_slider = widgets.FloatSlider(value=0, min=0, max=1, step=0.01, description='T:')
button = widgets.Button(description = 'Apply Spot Detection')

# Define callback function
def button_spot_detection(image):
    sigma = sigma_slider.value
    t = t_slider.value
    image = spot_detector(image, sigma, t)
    contours, _ = cv.findContours(image.astype(np.uint8), cv.RETR_LIST, cv.CHAIN_APPROX_NONE)
    print(f'Detected {len(contours):4} spots.', end='\r')
    return image

# Launch viewer
plt.close("all")
spot_detector_viewer = viewer(spots, title = "Spot Detector", new_widgets = [sigma_slider, t_slider, button], callbacks = [button_spot_detection], widgets=True, normalize=True )

### Multiple choice question (1 point)

What pair of parameters will give you exactly 6 spots? If there are more than one, try to select the most reasonable one. 

1. $\sigma_1 = 10$ and $T = 0.2$,
2. $\sigma_1 = 5$ and $T = 0.6$, 
3. $\sigma_1 = 5$ and $T = 0.2$, or
4. $\sigma_1 = 1$ and $T = 0.3$.

Modify the variable answer in the next cell to reflect your choice. Run the last cell to check that your answer is valid.

In [73]:
%use sos
# Assign your answer to this variable
answer = None
### BEGIN SOLUTION
answer = 4
### END SOLUTION

In [74]:
%use sos
# Sanity check
assert answer in [1, 2, 3, 4], 'Answer one of 1, 2, 3 or 4'
### BEGIN HIDDEN TESTS
assert answer == 2 or answer == 4
### END HIDDEN TESTS

Congratulations on finishing Lab 2!

Remember that, in doubt, you can always ask the TAs of the course. Now, make sure to save your notebook (keep a copy on your personal computer) and upload it to moodle.